In [11]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import folium
# !pip install pandas # 파이썬 데이터 처리
import pandas as pd

# 전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와서 지도 위에 표시하기

- requests 모듈로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.
- json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
- folium 모듈을 사용해서 표시하고 지도위에 스타벅스 매장 위치를 마커로 표시한다.

In [12]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=HBSCEBENZ1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' #?
})
starbucks = request.json()
#print(type(starbucks))
#print(starbucks)

In [13]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_nomalize(딕셔너리, '데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
starbucks_df = pd.json_normalize(starbucks, 'list')
print(type(starbucks_df)) # <class 'pandas.core.frame.DataFrame'>
print(len(starbucks_df))
print(starbucks_df.columns)

<class 'pandas.core.frame.DataFrame'>
1845
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p90', 'p01', 't05', 't30', 't36', 't27', 't29', 't43', 't48', 'z9999'],
      dtype='object', length=131)


데이터프레임으로 변환하니 서울에 있는 스타벅스 지점은 601개고 각 지점당 131개의 컬럼으로 정보를 표시하고 있다.
작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때, 사용할 데이터가 저장된 데이터프레임을 만든다.
s_name => 지점이름
sido_code => 시도코드
sido_name => 시도이름
gugun_code => 구군코드
 gugun_name => 구군이름
 doro_address => 도로명주소
 lat => 위도
 lot => 경도

In [14]:
# - 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적어도 되지만 얻어올 칼럼이 2개 이상일 경우
# 얻어올 컬럼들을 반드시 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[
    [
        's_name' ,
        'sido_code',
        'sido_name',
        'gugun_code',
        'gugun_name',
        'doro_address',
        'lat',
        'lot'
    ]
]
starbucks_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,현대동구5F,06,울산,0605,동구,울산광역시 동구 방어진순환도로 899 (서부동),35.521660,129.431357
1,미금역사거리,08,경기,0811,성남시 분당구,경기도 성남시 분당구 성남대로 151 (구미동),37.349605,127.108302
2,울산간절곶,06,울산,0603,울주군,울산광역시 울주군 서생면 대송리 25-18,35.3590048,129.3591978
3,송파마천사거리DT,01,서울,0114,송파구,서울특별시 송파구 거마로 78 (마천동),37.499829,127.147672
4,그랜드조선 부산,05,부산,0516,해운대구,부산광역시 해운대구 해운대해변로 292 (중동),35.1600338,129.1630512
...,...,...,...,...,...,...,...,...
1840,수원경수인계DT,08,경기,0815,수원시 팔달구,경기도 수원시 팔달구 경수대로 499 (인계동),37.270568,127.027566
1841,호계지식산업센터,08,경기,0819,안양시 동안구,경기도 안양시 동안구 엘에스로 122 (호계동) 118~123호,37.372351,126.948665
1842,안동이마트,11,경북,1123,안동시,경상북도 안동시 옥동1길 2 (옥동),36.558505,128.699308
1843,월계역,01,서울,0107,노원구,"서울특별시 노원구 초안산로2라길26 월계동(104,105,106호)",37.6295,127.0574


In [15]:
# dtype 속성으로 데이터 프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [16]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845 entries, 0 to 1844
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        1845 non-null   object
 1   sido_code     1845 non-null   object
 2   sido_name     1845 non-null   object
 3   gugun_code    1845 non-null   object
 4   gugun_name    1832 non-null   object
 5   doro_address  1845 non-null   object
 6   lat           1845 non-null   object
 7   lot           1845 non-null   object
dtypes: object(8)
memory usage: 115.4+ KB


In [17]:
# astype() 메소드로 위도, 경도 데이터 타입을 object에서 float으로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845 entries, 0 to 1844
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        1845 non-null   object 
 1   sido_code     1845 non-null   object 
 2   sido_name     1845 non-null   object 
 3   gugun_code    1845 non-null   object 
 4   gugun_name    1832 non-null   object 
 5   doro_address  1845 non-null   object 
 6   lat           1845 non-null   float64
 7   lot           1845 non-null   float64
dtypes: float64(2), object(6)
memory usage: 115.4+ KB


In [18]:
# 종각점
starbucsk_df_map[starbucsk_df_map['s_name'] == '종로관철']

NameError: name 'starbucsk_df_map' is not defined

In [19]:
starbucksMap = folium.Map(location=[37.56961146 , 126.9840133], zoom_start=17)
popup = folium.Popup('종각점',max_width=300)
folium.Marker(location=[37.569918,126.984528] , popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로관철',max_width=300)
folium.Marker(location=[37.569058,126.986013] , popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로구청',max_width=300)
folium.Marker(location=[37.572662,126.981138] , popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap1.html')
starbucksMap

In [20]:
# - 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도위에 표시한다.
# - 지도를 작성할 때 지도의 중심으로 설정할 위치가 애매할 경우 지도위에 표시할 모든 스타벅스 지점에 위도와 경도의 평균을 중심으로 하는 지도를 만들어 사용하면 된다.
# - 데이터프레임에 저장된 데이터의 평균은 maen() 메소드를 사용해 표시한다.
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean() , starbucks_df_map['lot'].mean()], zoom_start=17)

# iterrows() 메소드는 데이터프레임에 저장된 데이터의 인덱스와 데이터를 리턴한다.
for index, data in starbucks_df_map.iterrows():
    # print(index, data)
    popup = folium.Popup('{}. {} -{}'.format(index+1 , data['s_name'], data['doro_address']) , max_width=300)
    folium.Marker(location=[data['lat'],data['lot']] , popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap2.html')
starbucksMap

In [ ]:
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=HBSCEBENZ1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' #?
})
starbucks = request.json()
starbucks_df = pd.json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[ [ 's_name' ,   'sido_code',  'sido_name',  'gugun_code', 'gugun_name', 'doro_address', 'lat',  'lot' ]]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean() , starbucks_df_map['lot'].mean()], zoom_start=17)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} -{}'.format(index+1 , data['s_name'], data['doro_address']) , max_width=300)
    folium.Marker(location=[data['lat'],data['lot']] , popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
# starbucksMap

In [ ]:
# 전국 스타벅스 지도
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=HBSCEBENZ1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 1900, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' #?
})
starbucks = request.json()
starbucks_df = pd.json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[ [ 's_name' ,   'sido_code',  'sido_name',  'gugun_code', 'gugun_name', 'doro_address', 'lat',  'lot' ]]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean() , starbucks_df_map['lot'].mean()], zoom_start=8)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} -{}'.format(index+1 , data['s_name'], data['doro_address']) , max_width=300)
    folium.Marker(location=[data['lat'],data['lot']] , popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
# starbucksMap